# DSPY Learn - Modules

https://dspy.ai/learn/programming/modules/

In [1]:
#!pip install dspy

In [2]:
import dspy 
from common.my_settings import MySettings  
from common.utils import md

settings = MySettings().get()

#lm_gpt35 = dspy.LM('gpt-3.5-turbo', temperature=0.8, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
lm_gpt4omin = dspy.LM('gpt-4o-mini', temperature=0.9, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
dspy.configure(lm=lm_gpt4omin)

lm = lm_gpt4omin
lm('hello there')


Getting keys from environment variables


['Hello! How can I assist you today?']

In [3]:
sentence = "it's a charming and often affecting journey"
classify = dspy.Predict("sentence -> sentiment: bool")
response = classify(sentence=sentence)
print(response.sentiment)

True


In [4]:
question = "What's something great about the ColBERT retrieval module"
classify = dspy.ChainOfThought("question -> answer", n=5)
response = classify(question=question)
response.completions.answer

['One great aspect of the ColBERT retrieval module is its ability to efficiently combine the advanced contextual embeddings provided by BERT with a late interaction mechanism, enabling both high accuracy in document retrieval and reduced computational overhead during query processing.',
 "One great aspect of the ColBERT retrieval module is its combination of high retrieval effectiveness with efficient processing, allowing for fast similarity scoring using BERT's contextualized embeddings while managing large-scale search tasks effectively.",
 'One great aspect of the ColBERT retrieval module is its ability to perform effective late interaction, allowing it to efficiently match contextualized query and document representations, which leads to improved retrieval effectiveness while maintaining fast processing speeds.',
 'One great aspect of the ColBERT retrieval module is its ability to efficiently combine the rich semantic understanding of BERT with fast retrieval times, thanks to its l

In [5]:
print(f"Reasoning: {response.reasoning}")
print(f"Answer: {response.answer}")

Reasoning: ColBERT (Contextualized Late Interaction over BERT) is a retrieval module that effectively combines the benefits of contextual embeddings with efficient retrieval. One of its standout features is the ability to leverage the strengths of BERT's deep contextual representations while maintaining a highly efficient retrieval process. This is achieved through a two-stage process: first, it indexes documents with their token embeddings and then uses a late interaction mechanism to compute relevance scores during query time. This allows for a more nuanced understanding of queries and documents, leading to improved accuracy in retrieval tasks without incurring the high computational costs typically associated with full BERT inference.
Answer: One great aspect of the ColBERT retrieval module is its ability to efficiently combine the advanced contextual embeddings provided by BERT with a late interaction mechanism, enabling both high accuracy in document retrieval and reduced computat

# Math

Install Deno to run the Python code

In [6]:
# import os, pathlib, shutil, subprocess

# # Point to your Deno install (Codespaces default)
# deno_bin = str(pathlib.Path.home() / ".deno" / "bin")

# # Add to PATH for this kernel session
# os.environ["PATH"] = deno_bin + ":" + os.environ.get("PATH","")

# # (Optional) sanity checks
# print("deno path:", deno_bin)
# print("which deno:", shutil.which("deno"))
# subprocess.run(["deno","--version"], check=True)


deno path: /home/codespace/.deno/bin
which deno: /home/codespace/.deno/bin/deno


deno 2.4.5 (stable, release, x86_64-unknown-linux-gnu)
v8 13.7.152.14-rusty
typescript 5.8.3


CompletedProcess(args=['deno', '--version'], returncode=0)

In [9]:
maths = dspy.ProgramOfThought("question -> answer")
md(maths(question="Whats the volume of a circle that is 3cm in diameter?"))

Prediction(  
    reasoning='To find the volume of a sphere, we use the formula \\( V = \\frac{4}{3} \\pi r^3 \\), where \\( r \\) is the radius of the sphere. The diameter of the sphere is given as 3 cm, which means the radius \\( r \\) is half of the diameter: \\( r = \\frac{3}{2} = 1.5 \\) cm. Substituting this value into the volume formula gives us a calculated volume of approximately 14.14 cubic centimeters.',  
    answer='The volume of the sphere with a diameter of 3 cm is approximately 14.14 cm³.'  
)

In [18]:
maths.history

[{'prompt': None,
  'messages': [{'role': 'system',
    'content': 'Your input fields are:\n1. `question` (str):\nYour output fields are:\n1. `reasoning` (str): \n2. `generated_code` (str): python code that answers the question\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## question ## ]]\n{question}\n\n[[ ## reasoning ## ]]\n{reasoning}\n\n[[ ## generated_code ## ]]\n{generated_code}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        You will be given `question` and you will respond with `generated_code`.\n        Generating executable Python code that programmatically computes the correct `generated_code`.\n        After you\'re done with the computation and think you have the answer, make sure to provide your answer by calling the preloaded function `final_answer()`.\n        You should structure your answer in a dict object, like {"field_a": answer_a, ...}, evaluates to the correct value 

# RAG


In [24]:
lm('Whats the name of the Castle that David Gregory inherited?')

['David Gregory inherited the "Glenwood Castle."']

In [33]:
def search(query: str) -> list[str]:
    """Retreives abstracts from wikipedia"""
    # A COLBERT server is a vector database that can be used to store and retrieve embeddings.
    # The line below is fetching snippets from wikiped (2017) that is hosted on a COLBERT server. 
    result = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(query, k=3)

    md("**Search result**:", result[0])
    return [item['text'] for item in result]

rag = dspy.ChainOfThought("context, question -> response")
question = "Whats the name of the Castle that David Gregory inherited?"

md("**RAG with answerable question**")
md(rag(context=search(question), question=question))

# Uncomment the line below to see how it won't answer if the question cannot be answered from the context. 
# In the example below, the question is "Who is the president of Germany?" which cannot be answered from the context retrieved.
# it should give an answer such as "I don't know" or "The context does not provide this information."
md("**RAG with unanswerable question**")
md(rag(context=search(question), question="Who is the president of Germany?")) 

**RAG with answerable question**

**Search result**:{'text': 'David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinnairdy Castle in 1664. Three of his twenty-nine children became mathematics professors. He is credited with inventing a military cannon that Isaac Newton described as "being destructive to the human species". Copies and details of the model no longer exist. Gregory\'s use of a barometer to predict farming-related weather conditions led him to be accused of witchcraft by Presbyterian ministers from Aberdeen, although he was never convicted.', 'pid': 3296134, 'rank': 1, 'score': 25.856355667114258, 'prob': 0.9928459586069872, 'long_text': 'David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinnairdy Castle in 1664. Three of his twenty-nine children became mathematics professors. He is credited with inventing a military cannon that Isaac Newton described as "being destructive to the human species". Copies and details of the model no longer exist. Gregory\'s use of a barometer to predict farming-related weather conditions led him to be accused of witchcraft by Presbyterian ministers from Aberdeen, although he was never convicted.'}

Prediction(  
    reasoning='David Gregory, the Scottish physician mentioned in the context, inherited Kinnairdy Castle in 1664. The specific information about the castle is provided clearly in the first context entry.',  
    response='The name of the castle that David Gregory inherited is Kinnairdy Castle.'  
)

**RAG with unanswerable question**

**Search result**:{'text': 'David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinnairdy Castle in 1664. Three of his twenty-nine children became mathematics professors. He is credited with inventing a military cannon that Isaac Newton described as "being destructive to the human species". Copies and details of the model no longer exist. Gregory\'s use of a barometer to predict farming-related weather conditions led him to be accused of witchcraft by Presbyterian ministers from Aberdeen, although he was never convicted.', 'pid': 3296134, 'rank': 1, 'score': 25.856355667114258, 'prob': 0.9928459586069872, 'long_text': 'David Gregory (physician) | David Gregory (20 December 1625 – 1720) was a Scottish physician and inventor. His surname is sometimes spelt as Gregorie, the original Scottish spelling. He inherited Kinnairdy Castle in 1664. Three of his twenty-nine children became mathematics professors. He is credited with inventing a military cannon that Isaac Newton described as "being destructive to the human species". Copies and details of the model no longer exist. Gregory\'s use of a barometer to predict farming-related weather conditions led him to be accused of witchcraft by Presbyterian ministers from Aberdeen, although he was never convicted.'}

Prediction(  
    reasoning='The context provided contains information about historical figures named David Gregory and Gregory Tarchaneiotes, but it does not relate to current political figures or the presidency of Germany. Since the question is about the current president of Germany, I need to draw on my training data. As of my last update, the president of Germany is Frank-Walter Steinmeier, who has been in office since March 2017.',  
    response='The current president of Germany is Frank-Walter Steinmeier.'  
)